# ETL Pipeline: Uber Rides Analysis

This Jupyter notebook demonstrates a data engineering ETL pipeline using Python, Pandas, and SQLite. It processes NYC Uber ride data to extract insights like average fares by hour and day. The pipeline is modular, handles data quality, and produces visualizations for easy interpretation.

Dataset: uber_raw_data_sep14.csv (~4.5K rows, available from NYC TLC). Place it in the same directory.

Tech Stack:
- Extract: Pandas for CSV ingestion
- Transform: Clean data, derive metrics (e.g., ride hours, fares)
- Load: SQLite for storage
- Viz: Matplotlib/Seaborn for insights

Run all cells to execute the pipeline and see results!

## Step 1: Setup and Imports
Install dependencies: pip install pandas sqlalchemy matplotlib seaborn
Import libraries and configure visualization style.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set_style('whitegrid')
%matplotlib inline

## Step 2: Extract
Load the Uber ride data from CSV. Simulate real-world data ingestion (e.g., from API or cloud storage).

In [ ]:
# Read CSV file
try:
 df = pd.read_csv('uber_raw_data_sep14.csv')
 print(f'Extracted {len(df)} rows successfully')
 display(df.head())
except FileNotFoundError:
 print('Error: Place uber_raw_data_sep14.csv in the same directory.')
 raise

## Step 3: Transform
Clean the data, handle missing values, and derive new features (e.g., hour, day of week). Aggregate to compute metrics like average fare by hour and day.

In [ ]:
# Parse datetime and handle missing values
df['Date/Time'] = pd.to_datetime(df['Date/Time'])
df.dropna(subset=['Lat', 'Lon'], inplace=True)

# Add derived columns
df['Hour'] = df['Date/Time'].dt.hour
df['DayOfWeek'] = df['Date/Time'].dt.day_name()

# Simulate fare data (replace with real fares if available)
df['Fare'] = np.random.uniform(10, 50, len(df))

# Aggregate: Average fare and ride count by hour and day
agg_df = df.groupby(['Hour', 'DayOfWeek'])['Fare'].agg(['mean', 'count']).reset_index()
agg_df.rename(columns={'mean': 'Avg_Fare', 'count': 'Ride_Count'}, inplace=True)

print(f'Transformed data: {len(agg_df)} aggregated rows')
display(agg_df.head())

## Step 4: Load
Store the transformed data in a SQLite database for downstream analytics.

In [ ]:
# Create SQLite database and load data
engine = create_engine('sqlite:///uber_rides.db')
agg_df.to_sql('ride_metrics', engine, if_exists='replace', index=False)
print('Loaded aggregated data to SQLite table: ride_metrics')

# Verify by querying
check_df = pd.read_sql('SELECT * FROM ride_metrics LIMIT 5', engine)
display(check_df)

## Step 5: Visualize Insights
Create a bar plot of average fares by hour, colored by day of the week, to highlight trends.

In [ ]:
# Plot average fare by hour and day
plt.figure(figsize=(12, 6))
sns.barplot(data=agg_df, x='Hour', y='Avg_Fare', hue='DayOfWeek')
plt.title('Average Uber Fare by Hour and Day of Week')
plt.xlabel('Hour of Day')
plt.ylabel('Average Fare ($)')
plt.xticks(rotation=45)
plt.legend(title='Day of Week', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('fare_by_hour.png')
plt.show()

## Step 6: Summary
This ETL pipeline:
- Processed ~4.5K ride records into 168 aggregated rows.
- Handled missing data and derived actionable features.
- Stored results in a SQLite database for scalability.
- Visualized trends for business insights.

For CV: This notebook showcases data engineering skills like data cleaning, SQL integration, and visualization. Extend with Airflow for scheduling or AWS S3 for cloud storage to demonstrate advanced skills.